In [1]:
# POWER METHOD

import numpy as np
from scipy.linalg import eigh

def run_GSE(rho, H, observable, m=1):
    def normalize_density_matrix(rho):
        """Normalize the density matrix to have a trace of 1 and non-negative eigenvalues."""
        trace_rho = np.trace(rho)
        if trace_rho == 0:
            raise ValueError("Trace of density matrix rho is zero, cannot normalize.")
        eigenvalues, eigenvectors = np.linalg.eigh(rho)
        eigenvalues_clipped = np.clip(eigenvalues, 0, None)
        return eigenvectors @ np.diag(eigenvalues_clipped) @ np.conj(eigenvectors.T)

    rho_normalized = normalize_density_matrix(rho)
    
    def is_valid_density_matrix(rho_normalized):
        """Check if rho matrix is a valid density matrix."""
        return np.allclose(np.trace(rho_normalized), 1) and np.all(np.linalg.eigvals(rho) >= 0)

    if not is_valid_density_matrix(rho_normalized):
        raise ValueError("The normalized rho is still not a valid density matrix.")
    
    def is_hermitian(matrix):
        """Check if a matrix is Hermitian."""
        return np.allclose(matrix, matrix.conj().T)
    
    if not is_hermitian(H_matrix):
        raise ValueError("H_matrix is not Hermitian.")


    A = rho_normalized if m % 2 != 0 else np.eye(rho_normalized.shape[0])
    powers_of_rho_normalized = [np.linalg.matrix_power(rho_normalized, i) for i in range(m + 1)]
    

    #H_MATRIX NEEDS TO BE PASSED!!!

    H_matrix = np.zeros((m+1, m+1), dtype=np.complex128)

    S_matrix = np.zeros((m+1, m+1), dtype=np.complex128)

    for i in range(m + 1):
        for j in range(m + 1):
            sigma_i = powers_of_rho_normalized[i].conj().T
            sigma_j = powers_of_rho_normalized[j]
            H_matrix[i, j] = np.trace(sigma_i @ A @ sigma_j @ H)
            S_matrix[i, j] = np.trace(sigma_i @ A @ sigma_j)

    eigenvalues, eigenvectors = eigh(H_matrix, S_matrix)
    min_eigenvalue_index = np.argmin(eigenvalues)
    alpha = eigenvectors[:, min_eigenvalue_index]
    rho_EM = sum(alpha[i] * powers_of_rho_normalized[i] for i in range(m + 1))
    expectation_value = np.real(np.trace(rho_EM @ observable))
    
    return expectation_value



In [2]:
import numpy as np
from scipy.linalg import eigh

def run_GSE_power_subspace(rho, H, observable, max_power=0):
    """
    Run the GSE using a power subspace approach for each power up to max_power.

    Parameters:
    - rho: numpy.ndarray, the density matrix of the system.
    - H: numpy.ndarray, the system Hamiltonian.
    - observable: numpy.ndarray, the observable for which the expectation value is calculated.
    - max_power: int, the highest power of rho used to construct the subspace.

    Returns: 
    - A dictionary with keys being the power and values being the expectation value for each power.
    """
    def normalize_density_matrix(rho):
        """Normalize the density matrix to have a trace of 1 and non-negative eigenvalues."""
        trace_rho = np.trace(rho)
        if trace_rho == 0:
            raise ValueError("Trace of density matrix rho is zero, cannot normalize.")
        eigenvalues, eigenvectors = np.linalg.eigh(rho)
        eigenvalues_clipped = np.clip(eigenvalues, 0, None)
        return eigenvectors @ np.diag(eigenvalues_clipped) @ np.conj(eigenvectors.T)

    rho_normalized = normalize_density_matrix(rho)
    results = {}

    def is_valid_density_matrix(rho_normalized):
        """Check if rho matrix is a valid density matrix."""
        return np.allclose(np.trace(rho_normalized), 1) and np.all(np.linalg.eigvals(rho) >= 0)

    if not is_valid_density_matrix(rho_normalized):
        raise ValueError("The normalized rho is still not a valid density matrix.")
    
    def is_hermitian(matrix):
        """Check if a matrix is Hermitian."""
        return np.allclose(matrix, matrix.conj().T)
    
    if not is_hermitian(H):
        raise ValueError("H_matrix is not Hermitian.")

    for m in range(max_power + 1):
        # Generate powers of rho up to m
        powers_of_rho_normalized = [np.linalg.matrix_power(rho_normalized, i) for i in range(m + 1)]
        A = rho_normalized if m % 2 != 0 else np.eye(rho_normalized.shape[0])
        
        # Initialize Hamiltonian and Overlap matrices
        H_matrix = np.zeros((m+1, m+1), dtype=np.complex128)
        S_matrix = np.zeros((m+1, m+1), dtype=np.complex128)

        # Build the H and S matrices for subspace defined by powers up to m
        for i in range(m+1):
            for j in range(m+1):
                sigma_i = powers_of_rho_normalized[i].conj().T
                sigma_j = powers_of_rho_normalized[j]
                H_matrix[i, j] = np.trace(sigma_i @ A @ sigma_j @ H)
                S_matrix[i, j] = np.trace(sigma_i @ A @ sigma_j)

        #REGULARIZE S SUBSPACE
        epsilon = 1e-10
        np.fill_diagonal(S_matrix, np.diagonal(S_matrix) + epsilon)

        # Solve the generalized eigenvalue problem for this subspace
        eigenvalues, eigenvectors = eigh(H_matrix, S_matrix)
        min_eigenvalue_index = np.argmin(eigenvalues)
        optimal_alpha = eigenvectors[:, min_eigenvalue_index]

        # Construct the error-mitigated density matrix
        rho_EM = sum(optimal_alpha[k] * powers_of_rho_normalized[k] for k in range(m+1))
        expectation_value = np.real(np.trace(rho_EM @ observable))

        # Store the results for each power
        results[m] = expectation_value

    # Parse results to find the best result
    best_power = min(results, key=lambda k: abs(results[k]))
    best_expectation_value = results[best_power]
    print(f"Best result obtained for power {best_power}: {best_expectation_value}")
    return results


# Example usage
rho = np.array([[0.7, 0.3], [0.3, 0.3]])  # Example density matrix
H = np.array([[1, 0], [0, 0]])  # Example Hamiltonian
observable = np.array([[1, 0], [0, 0]])  # Example observable
max_power = 24 # Example to calculate using powers up to rho^2

results = run_GSE_power_subspace(rho, H, observable, max_power)
print(results)


Best result obtained for power 6: 0.0
{0: 0.7071067811688698, 1: 0.5962401709236329, 2: 7.275957614183426e-12, 3: 1.4551915228366852e-11, 4: -7.275957614183426e-12, 5: 2.2737367544323206e-11, 6: 0.0, 7: 9.094947017729282e-12, 8: -9.094947017729282e-13, 9: -1.0231815394945443e-12, 10: 9.094947017729282e-13, 11: -1.8616219676914625e-12, 12: 0.0, 13: -2.2737367544323206e-12, 14: 4.547473508864641e-12, 15: -3.1823432777855487e-12, 16: -1.0302869668521453e-11, 17: 1.4381384971784428e-11, 18: -5.8832938520936295e-12, 19: -2.8883562208648073e-12, 20: 6.30961949354969e-12, 21: 1.971756091734278e-12, 22: 8.952838470577262e-13, 23: 1.0231815394945443e-12, 24: 4.61142235508305e-12}


In [3]:
#FAULT SUBSPACE METHOD

import numpy as np
from scipy.linalg import eigh

def run_GSE_fault_subspace(rho, H, observable, noise_levels, noise_function, stretch_factor=1.0, regularization_factor=1e-5):
    """
    Run the GSE using a fault subspace approach with a customizable noise function and identity matrix A.

    Parameters:
    - rho: numpy.ndarray, the density matrix of the system.
    - H_matrix: numpy.ndarray, the system Hamiltonian.
    - observable: numpy.ndarray, the observable for which the expectation value is calculated.
    - noise_levels: list defining the noise levels or generating them dynamically.
    - noise_function: a function that applies noise to rho given noise levels.
    - stretch_factor: float, a factor to scale the noise levels, simulating stronger or weaker noise.
    - regularization_factor: float, a small value added to the diagonal of the S matrix for regularization.

    Returns:
    - The best expectation value and the corresponding noise level.
    """
    
    def is_valid_starting_density_matrix(rho):
        """Check if a matrix is a valid density matrix."""
        return np.allclose(np.trace(rho), 1) and np.all(np.linalg.eigvals(rho) >= 0)

    if not is_valid_starting_density_matrix(rho):
        raise ValueError("initial rho is not a valid density matrix.")
    
    def normalize_density_matrix(rho):
        """Normalize the density matrix to have a trace of 1 and non-negative eigenvalues."""
        trace_rho = np.trace(rho)
        if trace_rho == 0:
            raise ValueError("Trace of density matrix is zero, cannot normalize.")
        eigenvalues, eigenvectors = np.linalg.eigh(rho)
        eigenvalues_clipped = np.clip(eigenvalues, 0, None)
        return eigenvectors @ np.diag(eigenvalues_clipped) @ np.conj(eigenvectors.T)

    rho_normalized = normalize_density_matrix(rho)
    
    def is_valid_density_matrix(rho_normalized):
        """Check if a matrix is a valid density matrix."""
        return np.allclose(np.trace(rho_normalized), 1) and np.all(np.linalg.eigvals(rho_normalized) >= 0)

    if not is_valid_density_matrix(rho_normalized):
        raise ValueError("The normalized rho is still not a valid density matrix.")

    # Check H_matrix Hermitian 
    def is_hermitian(matrix):
        """Check if a matrix is Hermitian."""
        return np.allclose(matrix, matrix.conj().T)
    if not is_hermitian(H):
        raise ValueError("Input H is not Hermitian.")

    #A matrix set to identity as outlined in paper
    A = np.eye(rho_normalized.shape[0])

    results = []

    for level in noise_levels:
        # Applying noise and normalizing
        noisy_rho = noise_function(rho_normalized, level * stretch_factor)
        noisy_rho = normalize_density_matrix(noisy_rho)

        # Calculate H and S using the fault subspace approach
        H_matrix_noisy = np.zeros((len(noise_levels), len(noise_levels)), dtype=np.complex128)
        S_matrix_noisy = np.zeros((len(noise_levels), len(noise_levels)), dtype=np.complex128)

        for i in range(len(noise_levels)):
            for j in range(len(noise_levels)):
                sigma_i = normalize_density_matrix(noise_function(rho_normalized, noise_levels[i] * stretch_factor))
                sigma_j = normalize_density_matrix(noise_function(rho_normalized, noise_levels[j] * stretch_factor))

                cond_number = np.linalg.cond(S_matrix_noisy)
                print("Condition number of S matrix:", cond_number)
                min_eigval_S = np.min(np.linalg.eigvalsh(S_matrix_noisy))
                print("Minimum eigenvalue of S matrix:", min_eigval_S)

                H_matrix_noisy[i, j] = np.trace(sigma_i.conj().T @ sigma_j @ A @H)
                S_matrix_noisy[i, j] = np.trace(sigma_i.conj().T @ sigma_j @ A)

        # Regularize S matrix to ensure it is positive definite
        
        np.fill_diagonal(S_matrix_noisy, np.diagonal(S_matrix_noisy) + regularization_factor)
        eigvals = np.linalg.eigvalsh(S_matrix_noisy)
        if np.any(eigvals < 0):
            print("Adjusting S matrix due to negative eigenvalues.")
        regularization_factor = -np.min(eigvals) + 1e-8
        np.fill_diagonal(S_matrix_noisy, np.diagonal(S_matrix_noisy) + regularization_factor)
        cond_number = np.linalg.cond(S_matrix_noisy)
        print("Adjusted condition number of S matrix:", cond_number)


        # Solve the generalized eigenvalue problem for this subspace
        eigenvalues, eigenvectors = eigh(H_matrix_noisy, S_matrix_noisy)
        min_eigenvalue_index = np.argmin(eigenvalues)
        optimal_alpha = eigenvectors[:, min_eigenvalue_index]

        # Construct the error-mitigated density matrix
        rho_EM = sum(optimal_alpha[k] * normalize_density_matrix(noise_function(rho_normalized, noise_levels[k] * stretch_factor)) for k in range(len(noise_levels)))
        expectation_value = np.real(np.trace(rho_EM @ observable))

        # Store the results
        results.append((level, expectation_value))

    best_result = min(results, key=lambda x: abs(x[1]))
    return best_result
    
rho = np.array([[0.7, 0.3], [0.3, 0.3]])
H_matrix = np.array([[1, 0], [0, 1]])
observable = np.array([[1, 0], [0, 0]])
noise_levels = [0, 0.005, 0.0015, 0.0014, 0.006, 0.05, 0.1, 0.15, 0.2]

def noise_function(rho, noise_level):
    noise = noise_level * (np.random.normal(size=rho.shape) + 1j * np.random.normal(size=rho.shape))
    noisy_rho = rho + noise
    noisy_rho = (noisy_rho + noisy_rho.conj().T) / 2  # Symmetrize to ensure Hermitian
    eigvals, eigvecs = np.linalg.eigh(noisy_rho)
    eigvals[eigvals < 0] = 0  # Remove negative parts to maintain positive semi-definiteness
    return eigvecs @ np.diag(eigvals) @ eigvecs.conj().T

best_result = run_GSE_fault_subspace(rho, H_matrix, observable, noise_levels, noise_function)
print(f'Best noise level: {best_result[0]}, Best expectation value: {best_result[1]}')

Condition number of S matrix: inf
Minimum eigenvalue of S matrix: 0.0
Condition number of S matrix: inf
Minimum eigenvalue of S matrix: 0.0
Condition number of S matrix: inf
Minimum eigenvalue of S matrix: 0.0
Condition number of S matrix: inf
Minimum eigenvalue of S matrix: 0.0
Condition number of S matrix: inf
Minimum eigenvalue of S matrix: 0.0
Condition number of S matrix: inf
Minimum eigenvalue of S matrix: 0.0
Condition number of S matrix: inf
Minimum eigenvalue of S matrix: 0.0
Condition number of S matrix: inf
Minimum eigenvalue of S matrix: 0.0
Condition number of S matrix: inf
Minimum eigenvalue of S matrix: 0.0
Condition number of S matrix: inf
Minimum eigenvalue of S matrix: 0.0
Condition number of S matrix: inf
Minimum eigenvalue of S matrix: -0.4672404223893404
Condition number of S matrix: inf
Minimum eigenvalue of S matrix: -0.0019352740409980722
Condition number of S matrix: inf
Minimum eigenvalue of S matrix: -0.0019352740409980722
Condition number of S matrix: inf
Mi

In [4]:
# Example usage
def example_noise_model(rho, level):
    """Example noise model: Apply a simple depolarizing channel."""
    dim = rho.shape[0]
    identity = np.eye(dim) / dim
    return (1 - level) * rho + level * identity

rho = np.array([[0.7, 0.3], [0.3, 0.3]])  # Example density matrix, must be a valid quantum state
H = np.array([[1, 0], [0, 0]])  # Example Hamiltonian
observable = np.array([[1, 0], [0, 0]])  # Example observable
noise_levels = [0, 0.05, 0.1]  # Example noise levels

print('Error mitigated expectation value:', run_GSE_fault_subspace(rho, H, observable, noise_levels, example_noise_model))


Condition number of S matrix: inf
Minimum eigenvalue of S matrix: 0.0
Condition number of S matrix: inf
Minimum eigenvalue of S matrix: 0.0
Condition number of S matrix: inf
Minimum eigenvalue of S matrix: 0.0
Condition number of S matrix: inf
Minimum eigenvalue of S matrix: 0.0
Condition number of S matrix: inf
Minimum eigenvalue of S matrix: -0.45809844290512797
Condition number of S matrix: inf
Minimum eigenvalue of S matrix: 0.0
Condition number of S matrix: inf
Minimum eigenvalue of S matrix: 0.0
Condition number of S matrix: 8457.064150965227
Minimum eigenvalue of S matrix: -0.5845844620733582
Condition number of S matrix: 9273.498585685238
Minimum eigenvalue of S matrix: -0.5251068258250252
Adjusted condition number of S matrix: 220436544.38346976
Condition number of S matrix: inf
Minimum eigenvalue of S matrix: 0.0
Condition number of S matrix: inf
Minimum eigenvalue of S matrix: 0.0
Condition number of S matrix: inf
Minimum eigenvalue of S matrix: 0.0
Condition number of S mat